## TO DO:
- 

## Importing Libraries

In [1]:
import random
import numpy as np
import pandas as pd
import EagarTsai as et
import matplotlib.pyplot as plt
import seaborn as sns
import time
from rl import CustomEnv
import json
pd.set_option('display.max_rows', None)

## Initialising fixed parameters

In [2]:
"""Reading input file to get model parameter settings chosen by the user"""

f = open("model_params_train.json")
data = json.load(f)

# Assigning the values to variables so they can be used in the training and testing below

timestep = data['timestep']                 # Number of timesteps in an episode
max_steps = data['maximum_steps']           # Maximum number of timesteps across episodes in a single epoch
epochs = data['epochs']                     # Total number of epochs
model_alpha = data['model_alpha']           # Learning rate of the Bellmann equation
model_gamma = data['model_gamma']           # Discount factor of the Bellmann equation
model_epsilon = data['model_epsilon']       # Initial value for epsilon-greedy algorithm
store_data = bool(data['store_data'])       # Boolean value whether to store data locally or not

# The remaining varaibles dictate the minimum, maximum, and interval values for the three process parameters
min_power = data['min_power']
max_power = data['max_power']
interval_power = data['interval_power']
min_speed = data['min_speed']
max_speed = data['max_speed']
interval_speed = data['interval_speed']
min_hatch = data['min_hatch']
max_hatch = data['max_hatch']
interval_hatch = data['interval_hatch']

f.close()

## Creating parameter combinations

In [28]:
power = np.arange(min_power, max_power + 1, interval_power)
speed = np.arange(min_speed, max_speed + 1, interval_speed)
hatch = np.arange(min_hatch, max_hatch + 0.001, interval_hatch)

parameters = []
for p in power:
    for s in speed:
        for h in hatch:
            parameters.append((p,s,h))
            
len(parameters)

1650

## Running training model

#### The training model is where the model uses the epsilon greedy algorithm to explore the state space to find the state with the highest reward. The epsilon value comes into play and over time the agent begins exploiting more often, choosing the best action instead of a random one.

In [7]:
train_results = pd.DataFrame(columns = ['timesteps', 'test number', 'alpha', 'gamma', 'total steps', 'steps to optimal',\
                                     'optimal state', 'reward', 'reward per episode', 'time taken', 'no. of states visited'])

In [15]:
start_time = time.perf_counter()         # Beginning timer for the entire training

for j in range(0, epochs):               # For each epoch
    t1 = time.perf_counter()             # Beginning timer for the epoch
      
    # Creating instance of the environment
    env = CustomEnv(parameters, timestep, model_alpha, model_gamma, model_epsilon)
    
    episode = 0                          # Initialising episode number to 0
    states = set()                       # Total number of states visited in the epoch
    states_visited = []                  # Total number of states visited in the episode
    epoch_reward = 0                     # Total value of rewards in the epoch
    episode_rewards = []                 # Set containing each episode's total reward

    # As long as the maximum timesteps in an epoch is not exceeded
    while(env.steps < max_steps):
        env.reset(timestep)              # Resetting the environment
        done = False                     # Setting done to be False so the episode restarts
        episode += 1
        total_reward = 0                 # Episode reward initialised to zero before start of episode
        
        while not done:                  # For each episode
            reward, done = env.step(power, speed, hatch)
            total_reward += reward
            epoch_reward += reward
            states_visited.append(env.state)
            states.add(env.state)               
        
        print('Episode', episode, ' of epoch', j + 1, 'completed')
        #print('States visited in episode ', episode, 'of test', j+1, 'are ', states_visited)
        
        episode_rewards.append(total_reward)
        print('------------------')
        states_visited.clear()
    
    t2 = time.perf_counter()             # Ending timer for the epoch
    time_taken = t2 - t1                 # Calculating time taken for the epoch to run
    
    row = pd.Series([timestep, j + 1, env.alpha, env.gamma, env.steps, env.optimal_steps, env.optimal_state, \
                     env.rmax, epoch_reward / episode, time_taken, len(states)], \
                    index = train_results.columns)
    train_results.loc[len(train_results)] = row
    
    # If the user opts to store the results, then for each epoch the individual step results and the overall
    # train results will be stored in separate excel files.
    if store_data:
        env.results.to_excel(f'Train Results//Epoch {j + 1} Step Results.xlsx')
        train_results.sort_values('reward', ascending = False).to_excel(f'Train Results//Epoch {j + 1} Train Results.xlsx')
        
end_time = time.perf_counter()           # Ending timer for entire training

time_elapsed = end_time - start_time     # Calculating time taken for the whole training

Episode 1 completed
------------------
Episode 2 completed
------------------
Episode 3 completed
------------------


C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))
C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))
C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))
C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))
C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: in

Episode 4 completed
------------------
Episode 5 completed
------------------


C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))
C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))
C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))
C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))


Episode 1 completed
------------------
Episode 2 completed
------------------


...

Episode 3 completed
------------------
Episode 4 completed
------------------
Episode 5 completed
------------------
Episode 1 completed
------------------
Episode 2 completed
------------------


C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))
C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))


Episode 3 completed
------------------
Episode 4 completed
------------------


C:\Users\muadh\Documents\UoB\Research Assistant\Project\EagarTsai.py:136: RuntimeWarning: invalid value encountered in sqrt
  vx = (depth * width * (v ** 2)) / (4 * a * (1.5 * (depth + width / 2) - np.sqrt(depth * width / 2)))


Episode 5 completed
------------------


In [21]:
print('Elapsed Time is', time_elapsed / 60, 'minutes')

Elapsed Time is 4.407066221666669 minutes


In [22]:
"""Exporting results to an Excel sheet"""

#train_results = pd.read_excel('Tugrul Results.xlsx')

'Exporting results to an Excel sheet'

In [23]:
"""Importing results from an Excel sheet"""

#train_results.to_excel('Pre-Paper Results.xlsx', index = False)

'Importing results from an Excel sheet'

In [24]:
"""Displaying training details for each epoch"""

train_results.sort_values('reward', ascending = False)

,timesteps,test number,alpha,gamma,total steps,steps to optimal,optimal state,reward,reward per episode,time taken,no. of states visited
1,200.0,2.0,0.2,0.5,1000.0,376.0,1610.0,2.073876,56.982977,76.588603,652.0
2,200.0,3.0,0.2,0.5,1000.0,183.0,1610.0,2.073876,41.892755,75.494205,663.0
3,200.0,1.0,0.2,0.5,1000.0,51.0,1610.0,2.073876,49.482645,86.816980,636.0
5,200.0,3.0,0.2,0.5,1000.0,174.0,1610.0,2.073876,61.046617,95.442305,623.0
4,200.0,2.0,0.2,0.5,1000.0,242.0,1600.0,1.991684,58.715699,81.792709,673.0
0,200.0,1.0,0.2,0.5,1000.0,975.0,880.0,1.842630,18.194075,82.722810,616.0


In [27]:
"""Printing best parameter configuration from each epoch"""

print("Optimal parameter configurations (P, v, h):\n")
for i in range(len(train_results)):
    opt_state = train_results['optimal state'][i]
    print(f"Epoch {i + 1}: {parameters[int(opt_state)]}")

Optimal parameter configurations (P, v, h):

Epoch 1: (250, 750, 0.0375)
Epoch 2: (400, 2325, 0.0375)
Epoch 3: (400, 2325, 0.0375)
Epoch 4: (400, 2325, 0.0375)
Epoch 5: (400, 2100, 0.0375)
Epoch 6: (400, 2325, 0.0375)


In [14]:
"""Storing Q table as an Excel file if required for testing"""

# df = pd.DataFrame(data=env.qtable)

# df = (df.T)

# df.to_excel('Qtable_train.xlsx')

'Storing Q table as an Excel file if required for testing'

### Test Run

In [ ]:
"""If a specific Qtable is to be used, then import it here, otherwise the one from the train model is used"""
try:
    filename = "Qtable_train.xlsx"
    qtable = pd.read_excel(filename)
    print("Using imported Q-table")
except:
    qtable = env.qtable
    print("Using Q-table from train model")

In [ ]:
"""Reading input file to get model parameter settings chosen by the user for testing"""

f = open("model_params_test.json")
data = json.load(f)

# Assigning the values to variables so they can be used in the training and testing below

timestep = data['timestep']                 # Number of timesteps in an episode
max_steps = data['maximum_steps']           # Maximum number of timesteps across episodes in a single epoch
epochs = data['epochs']                     # Total number of epochs
model_alpha = data['model_alpha']           # Learning rate of the Bellmann equation
model_gamma = data['model_gamma']           # Discount factor of the Bellmann equation
model_epsilon = data['model_epsilon']       # Initial value for epsilon-greedy algorithm
store_data = bool(data['store_data'])       # Boolean value whether to store data locally or not

f.close()

In [ ]:
test_results = pd.DataFrame(columns = ['timestep', 'test number', 'alpha', 'gamma', 'total steps', 'steps to optimal',\
                                     'optimal state', 'reward', 'reward per episode', 'time taken', \
                                        'number of states'])

In [ ]:
# Creating a table to record only the reward for each episode
test_reward_table = pd.DataFrame(index = np.arange(1, 6, 1))
test_reward_table.index.name = 'Episode Number'

In [ ]:
start_time = time.perf_counter()         # Beginning timer for the entire training

for j in range(0,epochs):                # For each epoch
    t1 = time.perf_counter()             # Beginning timer for the epoch
    
    # Creating instance of the environment
    env_test = CustomEnv(parameters, timestep, model_alpha, model_gamma, model_epsilon, qtable = qtable)
    
    episode = 0                          # Initialising episode number to 0
    states = set()                       # Total number of states visited in the epoch
    states_visited = []                  # Total number of states visited in the episode
    epoch_reward = 0                     # Total value of rewards in the epoch
    episode_rewards = []                 # Set containing each episode's total reward

    # As long as the maximum timesteps in an epoch is not exceeded
    while(env_test.steps < max_steps):
        env_test.reset(timestep)         # Resetting the environment
        done = False                     # Setting done to be False so the episode restarts
        episode += 1                     
        total_reward = 0                 # Episode reward initialisd to zero before start of episode
        
        while not done:                  # For each episode
            reward, done = env_test.step(power, speed, hatch, test = True)
            total_reward += reward
            epoch_reward += reward
            states_visited.append(env_test.state)
            states.add(env_test.state)
        print('States visited in episode ', episode, 'of test', j+1, 'are ', states_visited)
        
        episode_rewards.append(total_reward)  
        print('------------------')
        states_visited.clear()
    
    t2 = time.perf_counter()             # Ending timer for the epoch
    time_taken = t2 - t1                 # Calculating time taken for the epoch to run
    
    test_reward_table[f'test {j + 1} rewards'] = episode_rewards
    row = pd.Series([timestep, j + 1, env_test.alpha, env_test.gamma, env_test.steps, env_test.optimal_steps,\
                     env_test.optimal_state, env_test.rmax, epoch_reward / episode, time_taken, \
                     len(states)], index = test_results.columns)
    test_results.loc[len(test_results)] = row
    
    # If the user opts to store the results, then for each epoch the individual step results and the overall
    # train results will be stored in separate excel files.
    if store_data:
        env.results.to_excel(f'Test Results//Epoch {j + 1} Step Results.xlsx')
        test_results.sort_values('reward', ascending = False).to_excel(f'Test Results//Epoch {j + 1} Test Results.xlsx')

    
end_time = time.perf_counter()           # Ending timer for entire training

time_elapsed = end_time - start_time     # Calculating time taken for the whole training

In [ ]:
print('Elapsed Time is', time_elapsed / 60, 'minutes')

In [ ]:
#test_reward_table.to_excel('Test Reward table 3.xlsx', index = False)

In [ ]:
test_results.sort_values('reward', ascending = False)

In [ ]:
"""Printing best parameter configuration from each epoch"""

print("Optimal parameter configurations (P, v, h):\n")
for i in range(test_results):
    opt_state = test_results['optimal state'][i]
    print(f"Epoch {i + 1}: {parameters[int(opt_state)]}")